Source Repo: https://github.com/wholidi/Project/tree/main/Minervini

In [ ]:
# Configurations

BASE_DATA_DIR = "yfinance_data"
END_DATE = "2025-09-18" # The end date is inclusive
TIME_PERIOD_DAYS = 365
INDEX_SYMBOL = "SPY"

In [ ]:
# Constants

TIME_FORMAT = "%Y-%m-%d"
FILE_TIME_FORMAT = TIME_FORMAT.replace("-", "_")

In [ ]:
# Helper Functions

import pandas as pd
import re

def get_file_date_str(date_str: str):
    date_dt = pd.to_datetime(date_str)
    return date_dt.strftime(FILE_TIME_FORMAT)

def normalize_ticker(ticker: str) -> str:
    """
    Normalize Wikipedia tickers to Yahoo Finance style:
    - Remove footnotes like [1], [a]
    - Convert BRK.B -> BRK-B, BF.B -> BF-B
    """
    ticker = re.sub(r"\[.*?\]", "", ticker)  # remove footnotes
    ticker = ticker.replace(".", "-")
    return ticker.strip()


In [ ]:
import pandas as pd
import requests
from io import StringIO

URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def get_sp500_tickers_on(date_str: str):
    """
    Return the list of S&P 500 tickers as of the given date (YYYY-MM-DD),
    and save full table with details to CSV.
    """

    # Fetch Wikipedia page
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }
    resp = requests.get(URL, headers=headers)
    resp.raise_for_status()

    # Read tables
    html_io = StringIO(resp.text)
    tables = pd.read_html(html_io, flavor="lxml")

    # --- Current constituents table ---
    df = tables[0]
    df['Symbol'] = df['Symbol'].astype(str).map(normalize_ticker)

    # Ensure all expected columns exist
    expected_cols = [
        "Symbol","Security","GICS Sector","GICS Sub-Industry",
        "Headquarters Location","Date added","CIK","Founded"
    ]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = ""

    # --- Changes table ---
    changes_df = None
    for table in tables:
        if isinstance(table.columns, pd.MultiIndex):
            table.columns = ["_".join([str(i) for i in col if i]).strip() for col in table.columns]
        cols_lower = [c.lower() for c in table.columns]
        if "ticker" in "".join(cols_lower) and "added" in "".join(cols_lower):
            changes_df = table.copy()
            break

    if changes_df is None:
        raise ValueError("Could not find the 'Selected changes to the list of S&P 500 components' table")

    date_col = [c for c in changes_df.columns if "date" in c.lower()][0]
    added_col = [c for c in changes_df.columns if "added" in c.lower() and "ticker" in c.lower()][0]
    removed_col = [c for c in changes_df.columns if "removed" in c.lower() and "ticker" in c.lower()][0]

    changes_df[date_col] = pd.to_datetime(changes_df[date_col], errors="coerce")
    changes_df = changes_df.dropna(subset=[date_col])
    changes_df = changes_df.sort_values(date_col, ascending=False)

    # Current tickers set
    tickers = set(df['Symbol'])

    target_date = pd.to_datetime(date_str)

    # Roll back changes after target_date
    for _, row in changes_df.iterrows():
        change_date = row[date_col]
        if change_date <= target_date:
            break

        added = normalize_ticker(str(row.get(added_col, "")))
        removed = normalize_ticker(str(row.get(removed_col, "")))

        if added and added in tickers:
            tickers.remove(added)
        if removed:
            tickers.add(removed)
            # Fill missing details for removed ticker if not in current df
            if removed not in df['Symbol'].values:
                new_row = {col: "" for col in expected_cols}
                new_row["Symbol"] = removed
                new_row["Security"] = str(row.get("Removed_Security", ""))
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    # Filter df to tickers that exist on target date
    final_df = df[df['Symbol'].isin(tickers)].copy()
    final_df = final_df[expected_cols]

    # 🔹 Save full table with sanitized tickers to CSV
    TICKERS_FILE = f"sp500_{get_file_date_str(date_str)}.csv"
    final_df.to_csv(TICKERS_FILE, index=False)
    print(f"✅ Saved full S&P 500 table with sanitized tickers to {TICKERS_FILE}")

    # 🔹 Return list of tickers
    return sorted(final_df['Symbol'].dropna().tolist())

In [ ]:
tickers = get_sp500_tickers_on(END_DATE)
print(f"S&P 500 tickers on {END_DATE}: {len(tickers)} symbols")
print(tickers[:20], "...")
# Check if PLTR is present in tickers
if "PLTR" in tickers:
    print("PLTR is present in tickers")
else:
    print("PLTR is not present in tickers on_date")

In [ ]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_yfinance_data(
    symbol: str, 
    start_date: str, 
    end_date: str, 
    interval: str = "1d", 
    save_to_file: bool = True
) -> pd.DataFrame:
    """
    Fetch historical OHLCV data for a given ticker from Yahoo Finance, 
    optionally using a cached file.

    Args:
        symbol (str): Ticker symbol (e.g., "AAPL", "SPY").
        start_date (str): Start date for historical data in 'YYYY-MM-DD' format.
        end_date (str): End date for historical data in 'YYYY-MM-DD' format.
        interval (str): Data interval ("1d", "1wk", "1mo", etc.). Default is "1d".
        save_to_file (bool): Whether to save/load data from cache. Default = True.

    Returns:
        pd.DataFrame: DataFrame with timestamp index and columns: open, high, low, close, volume.
                      Returns empty DataFrame if no data is found.
    """

    # 🔹 Always make `end_date` inclusive by adding +1 day
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)
    end_date_exclusive = end_dt.strftime("%Y-%m-%d")

    # 🔹 Normalize dates to use underscores instead of dashes
    start_clean = start_date.replace("-", "_")
    end_clean = end_date.replace("-", "_")

    # 🔹 Build directory name with dates + interval
    date_dir = f"{start_clean}_to_{end_clean}_{interval}"
    data_dir = os.path.join(BASE_DATA_DIR, date_dir)
    if save_to_file:
        os.makedirs(data_dir, exist_ok=True)

    file_path = os.path.join(data_dir, f"{symbol}.csv")  # 🔹 File name without date

    # Load from cache if available
    if save_to_file and os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        print(f"📄 Loaded {symbol} data from file ({len(df)} rows)")
        return df

    # Fetch live data
    print(f"Calling live API for {symbol}")
    ticker = yf.Ticker(symbol)
    df = ticker.history(start=start_date, end=end_date_exclusive, interval=interval)

    if df.empty:
        print(f"⚠️ No data found for {symbol}")
        return pd.DataFrame()

    df.index = df.index.tz_localize(None)  # remove timezone if present
    df = df.rename(columns={
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume"
    })

    df = df[["open", "high", "low", "close", "volume"]]

    # Save to cache if enabled
    if save_to_file:
        df.to_csv(file_path)
        print(f"✅ Saved {symbol} data to {file_path} ({len(df)} rows)")

    return df


In [ ]:
# from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Index return using Yahoo Finance (SPY as proxy for S&P 500)
end_date_dt = datetime.strptime(END_DATE, TIME_FORMAT) if END_DATE else datetime.now()
# start_date = (end_date_dt - timedelta(days=TIME_PERIOD_DAYS)).strftime(TIME_FORMAT)

# Subtract exactly 1 year (calendar-aware, handles leap years)
start_date_dt = (end_date_dt - relativedelta(years=1)) + timedelta(days=1)

start_date = start_date_dt.strftime(TIME_FORMAT)
end_date = end_date_dt.strftime(TIME_FORMAT)

print(start_date)
print(end_date)
index_df = get_yfinance_data(INDEX_SYMBOL, start_date=start_date, end_date=end_date)

# Ensure 'close' is float
index_df["close"] = index_df["close"].astype(float)

# Compute daily percent change
index_df["Percent Change"] = index_df["close"].pct_change()

# Compute cumulative return
index_return = (index_df["Percent Change"] + 1).cumprod().iloc[-1]

print(f"S&P 500 proxy return (SPY): {index_return:.2f}x")
print(index_df.head())

In [ ]:
def load_yahoo_data(tickers):
    """
    Fetch data once and store in yahoo_data dictionary.
    Writes passed and failed tickers to text files.
    """
    yahoo_data = {}
    successful_tickers = []
    failed_tickers = []

    for symbol in tickers:
        df = get_yfinance_data(symbol, start_date=start_date, end_date=end_date)

        if df.empty or "close" not in df.columns:
            print(f"⚠️ Skipping {symbol}: No data or bad format")
            failed_tickers.append(symbol)
            continue

        yahoo_data[symbol] = df
        successful_tickers.append(symbol)
        print(f"✅ Loaded {symbol} successfully")

    # Write failed tickers to file
    with open("failed_tickers.txt", "w") as f:
        for ticker in failed_tickers:
            f.write(f"{ticker}\n")

    # Write successful tickers to file (optional)
    with open("successful_tickers.txt", "w") as f:
        for ticker in successful_tickers:
            f.write(f"{ticker}\n")

    print(f"\n📄 {len(successful_tickers)} tickers loaded successfully and saved to successful_tickers.txt")
    print(f"📄 {len(failed_tickers)} tickers failed and saved to failed_tickers.txt")
    
    return yahoo_data, successful_tickers, failed_tickers


In [ ]:
def calculate_rs(index_return, start_date, end_date, index_symbol="SPY"):
    """
    Calculate RS scores using preloaded yahoo_data and precomputed index_return.
    Only processes successful tickers from load_yahoo_data().
    """
    global yahoo_data, successful_tickers

    returns_multiples = []

    for symbol in successful_tickers:
        if symbol == INDEX_SYMBOL:
            continue  # skip the index itself

        df_subset = yahoo_data[symbol].loc[start_date:end_date].copy()
        df_subset["close"] = df_subset["close"].astype(float)
        df_subset["Percent Change"] = df_subset["close"].pct_change()
        stock_return = (df_subset["Percent Change"] + 1).cumprod().iloc[-1]
        rs_score = stock_return / index_return
        returns_multiples.append(rs_score)
        print(f"✅ {symbol} processed – RS Score: {rs_score:.2f}")

    return returns_multiples

In [ ]:
import pandas as pd

def create_rs_top_df(successful_tickers, returns_multiples, top_quantile=0.7):
    """
    Create RS Rating DataFrame and filter top stocks by percentile.
    
    Args:
        successful_tickers (list): List of tickers successfully loaded.
        returns_multiples (list): Corresponding return multiples for tickers.
        top_quantile (float): Quantile threshold for top RS stocks (default 0.7).
    
    Returns:
        pd.DataFrame: Top RS stocks with columns ['Ticker', 'Returns_Multiple', 'RS_Rating'].
    """
    rs_df = pd.DataFrame({
        'Ticker': successful_tickers,
        'Returns_Multiple': returns_multiples
    })
    rs_df['RS_Rating'] = rs_df['Returns_Multiple'].rank(pct=True) * 100

    # Filter top stocks by RS_Rating quantile
    threshold = rs_df['RS_Rating'].quantile(top_quantile)
    rs_top_df = rs_df[rs_df['RS_Rating'] >= threshold].reset_index(drop=True)
    return rs_top_df

In [ ]:
import pandas as pd

def generate_minervini_report(yahoo_data, date_str, rs_top_df, export_prefix="Minervini"):
    """
    Generate Minervini report for top RS stocks without FutureWarning.
    """
    rows = []  # Collect row dicts here instead of using concat in the loop

    for stock in rs_top_df['Ticker']:
        try:
            df = yahoo_data.get(stock)
            if df is None or df.empty:
                print(f"⚠️ No data for {stock}, skipping")
                continue

            df.columns = [col.split(". ")[-1] for col in df.columns]
            df[['close', 'high', 'low']] = df[['close', 'high', 'low']].astype(float)

            df['SMA_50'] = df['close'].rolling(50).mean()
            df['SMA_150'] = df['close'].rolling(150).mean()
            df['SMA_200'] = df['close'].rolling(200).mean()

            currentClose = round(df['close'].iloc[-1], 2)
            SMA_50 = round(df['SMA_50'].iloc[-1], 2)
            SMA_150 = round(df['SMA_150'].iloc[-1], 2)
            SMA_200 = round(df['SMA_200'].iloc[-1], 2)
            low_52week = round(df['low'].iloc[-260:].min(), 2)
            high_52week = round(df['high'].iloc[-260:].max(), 2)

            Returns_Multiple = round(rs_top_df.loc[rs_top_df['Ticker'] == stock, 'Returns_Multiple'].iloc[0], 2)
            RS_Rating = round(rs_top_df.loc[rs_top_df['Ticker'] == stock, 'RS_Rating'].iloc[0])

            if pd.isnull([SMA_50, SMA_150, SMA_200]).any():
                status_msg = f"❌ Skipped {stock}: Not enough data for SMAs ({len(df)} rows)"
            else:
                SMA_200_20 = df['SMA_200'].iloc[-20] if len(df) >= 220 else 0
                conditions = [
                    currentClose > SMA_150 > SMA_200,
                    SMA_150 > SMA_200,
                    SMA_200 > SMA_200_20,
                    SMA_50 > SMA_150 > SMA_200,
                    currentClose > SMA_50,
                    currentClose >= 1.3 * low_52week,
                    currentClose >= 0.75 * high_52week
                ]
                if all(conditions):
                    status_msg = "✅ Passed Minervini"
                else:
                    failed = [str(i+1) for i, c in enumerate(conditions) if not c]
                    status_msg = f"❌ Failed Minervini conditions: {', '.join(failed)}"

            # Add row dict to list
            rows.append({
                'Stock': stock,
                'Price': currentClose,
                '50 Day MA': SMA_50,
                '150 Day MA': SMA_150,
                '200 Day MA': SMA_200,
                '52 Week Low': low_52week,
                '52 Week High': high_52week,
                'Returns_Multiple': Returns_Multiple,
                'RS_Rating': RS_Rating,
                'Status': status_msg
            })

        except Exception as e:
            print(f"⚠️ Could not process {stock}: {e}")

    # Create DataFrame once at the end
    detailedExportList = pd.DataFrame(rows)

    # Split and sort
    passed_df_sorted = detailedExportList[detailedExportList['Status'] == "✅ Passed Minervini"] \
        .sort_values(by=['Returns_Multiple', 'Stock'], ascending=[False, True]).reset_index(drop=True)

    failed_df_sorted = detailedExportList[detailedExportList['Status'] != "✅ Passed Minervini"] \
        .sort_values(by=['Returns_Multiple', 'Stock'], ascending=[False, True]).reset_index(drop=True)

    # Export
    date_file_str = get_file_date_str(date_str)
    passed_file = f"{export_prefix}_Passed_{date_file_str}.csv"
    failed_file = f"{export_prefix}_Failed_{date_file_str}.csv"

    passed_df_sorted.to_csv(passed_file, index=False)
    failed_df_sorted.to_csv(failed_file, index=False)

    print(f"\n📄 {len(passed_df_sorted)} stocks passed Minervini, saved to {passed_file}")
    print(f"📄 {len(failed_df_sorted)} stocks failed Minervini, saved to {failed_file}")


In [ ]:
yahoo_data, successful_tickers, failed_tickers = load_yahoo_data(tickers)

In [ ]:
returns_multiples = calculate_rs(index_return, start_date, end_date)
print(f"\nCalculated RS scores for {len(successful_tickers)} tickers.")

rs_top_df = create_rs_top_df(successful_tickers, returns_multiples)
generate_minervini_report(yahoo_data, END_DATE, rs_top_df)

In [ ]:
import pandas as pd

def get_valid_price(ticker: str, date_str: str) -> float | None:
    """
    Fetch the closing price for a ticker on a given date.
    Returns None if data is missing or invalid.
    """
    df = get_yfinance_data(ticker, date_str, date_str, save_to_file=False)
    if df.empty or "close" not in df.columns:
        print(f"⚠️ No data for {ticker} on {date_str}")
        return None

    price = df["close"].iloc[0]
    if pd.isna(price) or price <= 0:
        print(f"⚠️ Invalid price for {ticker} on {date_str}: {price}")
        return None

    return price

def backtest_minervini_from_file(passed_file, invest_date_str, exit_date_str, investment_per_stock=1000, interval="1d"):
    """
    Backtest Minervini strategy using previously saved passed tickers CSV.
    Invest `investment_per_stock` in each VALID ticker at END_DATE price and
    calculate returns 1 year later using get_yfinance_data().

    Only tickers with valid invest and exit prices are counted in total_invested.
    """
    passed_df = pd.read_csv(passed_file)
    if passed_df.empty:
        print("⚠️ No tickers in passed file.")
        return None

    total_portfolio_value = 0.0
    invested_count = 0
    invested_tickers = []

    for ticker in passed_df["Stock"]:
        try:
            # --- Get invest price ---
            invest_price = get_valid_price(ticker, invest_date_str)
            if invest_price is None:
                continue

            # --- Get exit price ---
            exit_price = get_valid_price(ticker, exit_date_str)
            if exit_price is None:
                continue

            # --- Compute final value and count the investment ---
            shares = investment_per_stock / invest_price
            final_value = shares * exit_price
            total_portfolio_value += final_value
            invested_count += 1
            invested_tickers.append(ticker)

            print(f"✅ {ticker}: Bought at {invest_price:.2f}, Sold at {exit_price:.2f}, Value: {final_value:.2f}")

        except Exception as e:
            print(f"⚠️ Error processing {ticker}: {e}")
            continue

    # Only count invested tickers
    total_invested = invested_count * investment_per_stock
    total_return_pct = ((total_portfolio_value / total_invested - 1) * 100) if total_invested > 0 else 0.0

    print("\n📊 Backtest Summary")
    print(f"Tickers in passed file: {len(passed_df)}")
    print(f"Tickers actually invested in: {invested_count}")
    print(f"Total invested: ${float(total_invested):,.2f}")
    print(f"Portfolio value after 1 year: ${float(total_portfolio_value):,.2f}")
    print(f"Total return: {float(total_return_pct):.2f}%")

    return {
        "invested_count": invested_count,
        "invested_tickers": invested_tickers,
        "total_invested": int(total_invested),  # safe since it's always multiple of $1,000
        "portfolio_value": round(float(total_portfolio_value), 2),
        "total_return_pct": round(float(total_return_pct), 2)
    }


In [ ]:
backtest_minervini_from_file("Minervini_Passed_2025_09_18.csv", start_date, end_date)